In [12]:
import sys
sys.path.append("../src/")
#import analysis

In [18]:
import os

# Den aktuellen Arbeitsverzeichnispfad erhalten
current_directory = os.getcwd()

print("Der aktuelle Pfad ist:", current_directory)

# Aktuellen Arbeitsverzeichnispfad ändern
neuer_pfad = 'C:/Users/domin/Desktop/Master/Praktikum_EI2/Microgrid_EnergyLab/Microgrid_EnergyLab'

os.chdir(neuer_pfad)

current_directory = os.getcwd()
print("Der aktuelle Pfad ist:", current_directory)

Der aktuelle Pfad ist: C:\Users\domin\Desktop\Master\Praktikum_EI2\Microgrid_EnergyLab
Der aktuelle Pfad ist: C:\Users\domin\Desktop\Master\Praktikum_EI2\Microgrid_EnergyLab\Microgrid_EnergyLab


In [20]:
from gboml import GbomlGraph

model_txt = "benchmark.txt"
gboml_model = GbomlGraph(8760)
nodes, edges, _ = gboml_model.import_all_nodes_and_edges("../models/scenarios/benchmark.txt")
gboml_model.add_nodes_in_model(*nodes)
gboml_model.add_hyperedges_in_model(*edges)
gboml_model.build_model()
solution = gboml_model.solve_gurobi(details=True)
#solution, solution_dict  = analysis.run_model(model_txt)
print(solution)

No such file as ../models/scenarios/benchmark.txt


NameError: name 'exit' is not defined

In [ ]:
solution_dict["solution"]["elements"]["TEST_NODE"]

In [ ]:
#analysis.show_results(solution_dict)

In [ ]:
import pandas as pd
results_week = pd.DataFrame()
elements = solution_dict["solution"]["elements"]
for element in elements.keys():
    if element in ["WIND_PLANT", "SOLAR_PV", "DEMAND_EL", "DISTRIBUTION_EL", "BATTERY", "CHP_PLANT"]:
        for variable in elements[element]["variables"].keys():
            var_values = elements[element]["variables"][variable]["values"]
            if len(var_values) == 1:
                result = f"Node {element}: {variable}: {var_values[0]}"
                print(result)
            else:
                col = variable
                results_week[col] = var_values[: 24*7]
                print(f"Node {element}: {variable}: (first week)")
                #plt.plot(var_values[: 24 * 7])
                #plt.show()

In [ ]:
results_week.head()

In [ ]:
results = results_week[96:120].reset_index(drop=True)
results.head()

In [ ]:
config = {'benchmark.txt': 10,
'model.txt': 10,
'model_100.txt':100,
'model_1000.txt': 1000, 
}
print(config)

In [ ]:
obj_result, nodes_result = analysis.get_results_new(config)
obj_result.to_csv("obj_result.csv")
nodes_result.to_csv("nodes_result.csv")


In [ ]:
obj_result.head()

In [ ]:
nodes_result.head()

In [ ]:
solution_dict['solution']["elements"].keys()

In [ ]:
solution_dict['solution']["elements"]["DISTRIBUTION_EL"]["objectives"]["unnamed"]

In [ ]:
objective_res, node_res = get_results(solution_dict, "", num_households=10)
objective_res.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(24)
plt.rcParams["figure.figsize"] = 9, 6

plt.plot(x, results.consumption_el)
plt.plot(x, results.electricity_import)
plt.plot(x, results.wind_generated)
plt.plot(x, results.solar_generated)
plt.plot(x, results.electricity_generated)

plt.xlabel("Hour")
plt.ylabel("kW")
plt.legend(['demand', 'electricity_import', 'wind_generated', 'solar_generated', "chp_electricity"], loc='upper left')
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = 9, 6

plt.plot(x, results.consumption_el)
plt.plot(x, results.electricity_import)
plt.plot(x, results.solar_generated+results.wind_generated)
plt.xlabel("Hour")
plt.ylabel("kW")
plt.legend(['demand', 'electricity_import', 'solar + wind'], loc='upper left')
plt.show()


In [ ]:
results["operating_cost_electricity"]

In [ ]:
ax = results[[ 'wind_generated', 'solar_generated', 'electricity_import', 'electricity_generated']].plot(kind='bar', stacked=True, figsize=(9, 6), rot=0, xlabel='Hour', ylabel='kW')
ax2 = results.operating_cost_electricity.plot(secondary_y=True,color="grey", ax=ax, label="operating cost electricity")
_ = ax.plot(results["consumption_el"],  color='black',  label='Demand')
_ = ax.legend()
_ = ax2.legend(loc="upper right")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13, 6))
_ = axes[0].plot(x, results["soc"], label="soc")
_ = axes[1].plot(x, results["discharge"], color="grey", label="discharge")
_ = fig.legend()